# Tutorial Week 12: Time-series analysis and Periodograms

This Workbook is a short instroduction to using the periodogram code. You can use the GLS code for your assignment

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

#from GLS import GLS as GLS

Below is a code for that will generate the Generalised Lomb Scargle Periodogram. 

**Exercise 1:** Look through the code and annotate it.

In [ ]:
#Routine to calculate the Generalised Lomb Scargle (see Zechmeister & Kuster 2009).
#Inputs:
# t - time 
# y - data values
# sig - uncertainties on y
# period - array containing the periods on which the periodogram should be calculated.
#
#Outputs:
# p_om - the periodogram
# a_om - the amplitude of the cosine term 
# b_om - the amplitude of the sine term
# Note: total amplitude is sqrt(a_om+b_om)
def GLS(t,y,sig,period=np.logspace(-1.,1.4,int(1e5))):
    W=np.sum(1./sig**2)
    w=1./W * 1./sig**2

    Y=np.sum(w*y)
    YYh=np.sum(w*y*y)
    p_om=np.zeros(period.shape[0])
    a_om=np.zeros(period.shape[0])
    b_om=np.zeros(period.shape[0])
    for i,P in enumerate(period):
        omt=2.*np.pi*t/P
        cos_omt=np.cos(omt)
        sin_omt=np.sin(omt)
        C=np.sum(w*cos_omt)
        S=np.sum(w*sin_omt)
        YCh=np.sum(w*y*cos_omt)
        YSh=np.sum(w*y*sin_omt)
        CCh=np.sum(w*cos_omt*cos_omt)
        SSh=np.sum(w*sin_omt*sin_omt)
        CSh=np.sum(w*cos_omt*sin_omt)
        YY=YYh-Y*Y
        YC=YCh-Y*C
        YS=YSh-Y*S
        CC=CCh-C*C
        SS=SSh-S*S
        CS=CSh-C*S
        D=CC*SS-CS**2
        p_om[i]=(SS*YC**2+CC*YS**2-2*CS*YC*YS)/(YY*D)
        a_om[i]=(YC*SS-YS*CS)/D
        b_om[i]=(YS*CC-YC*CS)/D
    return p_om,a_om,b_om

Simulate some a simple sinusoidal curve both with two different noise levels to test the GLS code. The sinusoidal curve contains a phase offset of pi/4 (45 degrees) and is uniformly sampled (if you want, you can also test the FFT method on this data).

In [ ]:
t=np.arange(0,30.,0.01)
Per=4.5
mdl=np.sin(2.*np.pi*t/Per+np.pi/4)
mdl_noise1=mdl+np.random.normal(0.,0.1,t.shape[0])
mdl_noise2=mdl+np.random.normal(0.,5.,t.shape[0])
sig1=t*0.+0.1
sig2=t*0.+5.
plt.plot(t,mdl_noise2,'k.')
plt.plot(t,mdl,linewidth=3)



**Exercise 2:** Calculate the periodogram both for the model with and without noise. For a useful period grid, you want to use something that is uniform in frequency (or at the very least uniform in logspacing, using np.logspace). The longest period/lowest frequency that you want to use should equal the total timespan / 2 (for period) or the 1/(2 time the timespan) (for the frequency). You also want to oversample the periods to get a nice smooth periodogram. You can use the P below as a starting point.



In [ ]:
P=np.logspace(-1.3,np.log10(30./2.),int(1e5))
p_om,a_om,b_om=GLS(t,mdl_noise2,sig2,P)



**Exercise 3:** After calculating the periodogram for the two noise-levels, try to use the periodogram to find the best fit model for the strongest period. Subtract this model from the data and reconstruct the periodogram. The model should be given by a($\omega_{best})$cos($\omega_{best}t$)+b($\omega_{best})$sin($\omega_{best}t$))

We will now generate some non-uniformly sampled data. First, we use a random number generator to select about 1 in 5 points from the overall sample. We sort this array, and only select the unique values, discarding any duplicates. Finally, we us np.hstack to cut chuncks out of the idx array

In [ ]:
#Generate a random sampling of 1/5th the points and sort the array
time_sampling=np.random.randint(0,t.shape[0],int((t.shape[0]/5)))
time_sampling.sort()

#Discard any duplicate points
idx=np.unique(time_sampling)

#Cut out two larger chunks, in this case from 70 to 110 and 220 to 240
idx=np.hstack( (idx[0:70],idx[110:220],idx[240:]))

#define the new time and data & sigma arrays.
t_sample=t[idx].copy()
mdl_noise2_sample=mdl_noise2[idx].copy()
sig2_sample=sig2[idx].copy()

plt.plot(t_sample,mdl_noise2_sample,'k.')
plt.plot(t,mdl,linewidth=3)

**Exercise 4** Calculate the periodogram for this dataset with gaps in it

To test the impact of the sampling on the data, we can simulate pure white noise with similar noise properties as the data, and recreate the periodogram. Any peaks that are in the same place for the white noise simulations and the real data are likely due to the way we sampled the data, and not due to any intrinsic periodicity that we are interested in. Note that although the locations of the peak in the white noise data set can match those in the data, their amplitudes will be different and cannot be directly compared.

In [ ]:
#Generate a white-noise dataset and make a periodogram
mockdata=np.random.normal(t_sample*0.,sig2_sample)



**Exercise 5:** Calculate the periodogram for the white-noise data and compare it to the one for the simulated data

Finally, let's look at a more complicated model with multiple frequencies and phase shifts.

In [ ]:
Per1=1.2
Per2=6.265
Per3=11.
mdl=np.sin(2.*np.pi*t/Per1+np.pi/4)+0.75*np.sin(2.*np.pi*t/Per2+np.pi/8)-1.3*np.sin(2.*np.pi*t/Per3+3*np.pi/4)
mdl_noise2=mdl+np.random.normal(0.,5.,t.shape[0])
t_sample=t[idx].copy()
mdl_noise2_sample=mdl_noise2[idx].copy()
sig2_sample=sig2[idx].copy()

plt.plot(t_sample,mdl_noise2_sample,'k.')
plt.plot(t,mdl,linewidth=3)

**Exercise 6:** Create the periodogram, and then try to subtract the strongest peak. Repeat this, until you think that you are no longer removing valid peaks from the data.